In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import os
#print(os.listdir("../input"))

import time

import matplotlib.pyplot as plt
import seaborn as sns
import gc

%matplotlib inline
path='/Users/alessandrorivello/Kaggle/dados_kaggle/'

In [2]:
# Setting the data types before importing (instead of letting pandas guess it) can improve 
# performance and use less RAM
# Reference: https://www.kaggle.com/yuliagm/how-to-work-with-big-datasets-on-16g-ram-dask
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'days'          : 'uint8',
        'hours'         : 'uint8',
        'minutes'       : 'uint8',
        'seconds'       : 'uint8',
        }

In [3]:
# We delete the attributed_time column since it has a lot of missing values and might not be so informative
# Reference: https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/51411
cols = ['ip', 'app', 'device', 'os', 'channel', 'is_attributed', 'days', 'hours', 'minutes','seconds']

In [10]:
# Note that we're using 10_train_timed and not the original file. This file represents 10% of the full train data
# and the column click_time was split into its components days, hours, minutes and seconds. Besides that, we also
# deleted the year and month component saving us a lot of memory space

train=pd.read_csv(path + '10_train_timed.csv', dtype=dtypes, usecols = cols)

In [13]:
dtypes2 = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'days'          : 'uint8',
        'hours'         : 'uint8',
        'minutes'       : 'uint8',
        'seconds'       : 'uint8',
        }

test=pd.read_csv(path + 'test_timed.csv', index_col=0, dtype=dtypes2)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
test.head()

,ip,app,device,os,channel,days,hours,minutes,seconds
click_id,,,,,,,,,
0,5744,9,1,3,107,10,4,0,0
1,119901,9,1,3,466,10,4,0,0
2,72287,21,1,19,128,10,4,0,0
3,78477,15,1,13,111,10,4,0,0
4,123080,12,1,13,328,10,4,0,0


In [15]:
gc.collect()

442

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18490388 entries, 0 to 18490387
Data columns (total 10 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
is_attributed    uint8
days             uint8
hours            uint8
minutes          uint8
seconds          uint8
dtypes: uint16(4), uint32(1), uint8(5)
memory usage: 299.8 MB


In [17]:

test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 0 to 18790468
Data columns (total 9 columns):
ip         uint32
app        uint16
device     uint16
os         uint16
channel    uint16
days       uint8
hours      uint8
minutes    uint8
seconds    uint8
dtypes: uint16(4), uint32(1), uint8(4)
memory usage: 430.1 MB
